In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter

torch.set_printoptions(linewidth=120)

In [2]:
train_set = torchvision.datasets.FashionMNIST(
    root= './data',
    download= True,
    train= True,
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
)

In [17]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [5]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 12*4*4)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        
        return t

network = Network()

In [7]:
network.parameters

<bound method Module.parameters of Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)>

In [18]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)
optimizer = optim.Adam(network.parameters(), lr=0.01)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter("runs/FashionMNIST/trained model")
tb.add_image("FashionMNIST Images", grid)
tb.add_graph(network, images)

for epoch in range(10):
    total_loss = 0
    total_correct = 0

    for batch in train_loader:
        images, labels = batch
        preds = network(images)
        loss = F.cross_entropy(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    tb.add_scalar("Loss", total_loss, epoch)
    tb.add_scalar("Number Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct/len(train_set), epoch)

    tb.add_histogram("conv1.bias", network.conv1.bias, epoch)
    tb.add_histogram("conv1.weight", network.conv1.weight, epoch)
    tb.add_histogram("conv1.weight.grad", network.conv1.weight.grad, epoch)

    print(f"epoch {epoch} total_loss {total_loss:.3f} total_correct {total_correct}" )

tb.close()

epoch 0 total_loss 351.918 total_correct 46539
epoch 1 total_loss 233.637 total_correct 51354
epoch 2 total_loss 213.605 total_correct 52099
epoch 3 total_loss 206.150 total_correct 52398
epoch 4 total_loss 199.921 total_correct 52623
epoch 5 total_loss 196.935 total_correct 52703
epoch 6 total_loss 191.175 total_correct 52982
epoch 7 total_loss 190.049 total_correct 53056
epoch 8 total_loss 188.924 total_correct 53032
epoch 9 total_loss 182.076 total_correct 53285


In [19]:
for name, weigth in network.named_parameters():
    print(name, "\t", weigth.shape)

conv1.weight 	 torch.Size([6, 1, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([12, 6, 5, 5])
conv2.bias 	 torch.Size([12])
fc1.weight 	 torch.Size([120, 192])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([60, 120])
fc2.bias 	 torch.Size([60])
out.weight 	 torch.Size([10, 60])
out.bias 	 torch.Size([10])


In [22]:
for name, weight in network.named_parameters():
    print(f"{name}.grad", "\t", weight.grad.shape)

conv1.weight.grad 	 torch.Size([6, 1, 5, 5])
conv1.bias.grad 	 torch.Size([6])
conv2.weight.grad 	 torch.Size([12, 6, 5, 5])
conv2.bias.grad 	 torch.Size([12])
fc1.weight.grad 	 torch.Size([120, 192])
fc1.bias.grad 	 torch.Size([120])
fc2.weight.grad 	 torch.Size([60, 120])
fc2.bias.grad 	 torch.Size([60])
out.weight.grad 	 torch.Size([10, 60])
out.bias.grad 	 torch.Size([10])


In [29]:
batch_size = 100
lr = 0.01

network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
optimizer = optim.Adam(network.parameters(), lr=lr)

images, lables = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

comment = f"batch_size = {batch_size} lr = {lr}"
tb = SummaryWriter("runs/FashionMNIST", comment=comment)
tb.add_image("FashionMNIST Images",grid)
tb.add_graph(network, images)

for epoch in range(10):
    total_loss = 0
    total_correct = 0

    for batch in train_loader:
        images, labels = batch
        
        preds = network(images)
        loss = F.cross_entropy(preds, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch_size # if we use different batch
        total_correct += get_num_correct(preds, labels)

    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar("Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct/len(train_set), epoch)

    # tb.add_histogram("conv1.bias", network.conv1.bias, epoch)
    # tb.add_histogram("conv1.weight", network.conv1.weight, epoch)
    # tb.add_histogram("conv1.weight.grad", network.conv1.weight.grad, epoch)

    for name, weight in network.named_parameters():
        tb.add_histogram(name, weight, epoch)
        tb.add_histogram(f"{name}.grad", weight.grad, epoch)

    print(f"epoch {epoch} total_correct {total_correct} total_loss {total_loss:.3f}")

tb.close()

epoch 0 total_correct 47494 total_loss 334.805
epoch 1 total_correct 51441 total_loss 234.553
epoch 2 total_correct 52115 total_loss 214.687
epoch 3 total_correct 52540 total_loss 202.426
epoch 4 total_correct 52882 total_loss 192.749
epoch 5 total_correct 52940 total_loss 192.185
epoch 6 total_correct 53008 total_loss 188.406
epoch 7 total_correct 53197 total_loss 185.579
epoch 8 total_correct 53182 total_loss 183.054
epoch 9 total_correct 53337 total_loss 182.899


## Looping using different parameters values

In [37]:
# parameters list
batch_size_list = [100, 1000, 10000]
lr_list = [0.01, 0.001, 0.0001]

# nasted iteration
for batch_size in batch_size_list:
    for lr in lr_list:
        network = Network()
        train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
        optimizer = optim.Adam(network.parameters(), lr=lr)
        for epoch in range(1):
            for batch in train_loader:
                break
                # images, labels = next(iter(train_loader))
                """
                next codes
                """

### Better option for doing the same this

In [38]:
from itertools import product
parameters = dict(
    lr = [0.01,0.001],
    batch_size = [100,1000,10000],
    shuffle = [True, False]
)
param_values = [v for v in parameters.values()]
param_values

[[0.01, 0.001], [100, 1000, 10000], [True, False]]

In [39]:
for lr, batch_size, shuffle in product(*param_values): # *arg will accept a turple insted of a single value
    print(lr, batch_size, shuffle)

0.01 100 True
0.01 100 False
0.01 1000 True
0.01 1000 False
0.01 10000 True
0.01 10000 False
0.001 100 True
0.001 100 False
0.001 1000 True
0.001 1000 False
0.001 10000 True
0.001 10000 False


In [44]:
network = Network()

for lr, batch_size, shuffle in product(*param_values):
    print(lr, batch_size, shuffle)
    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=shuffle)
    optimizer = optim.Adam(network.parameters(), lr=lr)

    images, lables = next(iter(train_loader))
    grid = torchvision.utils.make_grid(images)

    comment = f"batch_size={batch_size} lr={lr} shaffle={shuffle}"
    
    tb = SummaryWriter(comment=comment)
    tb.add_image("FashionMNIST Images",grid)
    tb.add_graph(network, images)

    for epoch in range(10):
        total_loss = 0
        total_correct = 0

        for batch in train_loader:
            images, labels = batch
        
            preds = network(images)
            loss = F.cross_entropy(preds, labels)
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * batch_size # if we use different batch
            total_correct += get_num_correct(preds, labels)

        tb.add_scalar('Loss', total_loss, epoch)
        tb.add_scalar("Correct", total_correct, epoch)
        tb.add_scalar("Accuracy", total_correct/len(train_set), epoch)

        # tb.add_histogram("conv1.bias", network.conv1.bias, epoch)
        # tb.add_histogram("conv1.weight", network.conv1.weight, epoch)
        # tb.add_histogram("conv1.weight.grad", network.conv1.weight.grad, epoch)

        for name, weight in network.named_parameters():
            tb.add_histogram(name, weight, epoch)
            tb.add_histogram(f"{name}.grad", weight.grad, epoch)

        print(f"epoch {epoch} total_correct {total_correct} total_loss {total_loss:.3f}")

tb.close()

0.01 100 True
epoch 0 total_correct 46629 total_loss 35281.992
epoch 1 total_correct 51106 total_loss 23920.710
epoch 2 total_correct 51802 total_loss 22130.772
epoch 3 total_correct 52184 total_loss 21252.583
epoch 4 total_correct 52372 total_loss 20496.213
epoch 5 total_correct 52599 total_loss 20108.388
epoch 6 total_correct 52644 total_loss 19893.620
epoch 7 total_correct 52730 total_loss 19610.529
epoch 8 total_correct 52884 total_loss 19135.154
epoch 9 total_correct 52907 total_loss 19092.968
0.01 100 False
epoch 0 total_correct 52932 total_loss 19290.478
epoch 1 total_correct 53110 total_loss 18676.609
epoch 2 total_correct 53001 total_loss 19182.474
epoch 3 total_correct 53324 total_loss 18116.596
epoch 4 total_correct 53336 total_loss 18178.562
epoch 5 total_correct 53252 total_loss 18549.639
epoch 6 total_correct 53374 total_loss 17878.157
epoch 7 total_correct 53456 total_loss 17947.817
epoch 8 total_correct 53397 total_loss 18068.058
epoch 9 total_correct 53486 total_loss 1